# 0. 공통

In [1]:
# !pip install tqdm

In [10]:
# Import pacakge
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup as bs
from tqdm import tqdm # 루프(반복문)의 진행 상황을 시각적으로 보여주는 라이브러리
import re
import numpy as np

# 접속 정보
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36"
}

In [ ]:
# dc 인사이드 갤러리 id

# 국내주식갤
# gal_id = "krstock"
# 4300이 최대페이지 / 2025년 5월 24일까지 가능

# 코스피갤
# gal_id = "kospi"
# 11000 페이지 - 2024년 12월 11일 (넉넉)

# 주식갤
# gal_id = "neostock"
# 10000 페이지 - 2024년 12월 17일 (넉넉)

# 삼성전자갤 - 근데 주식 아닌 것 까지 있음
# gal_id = "samsungelectron"
# 7이 최대페이지 / 2023년 6월 23일까지

# 1. 특정 갤러리 내 1페이지 리스트 크롤링 테스트

In [ ]:
gal_id = "kospi"
page_no = 11000 # 2024년 12월 11일 (넉넉)

url = f"https://gall.dcinside.com/mgallery/board/lists/?id={gal_id}&page={page_no}"

response = requests.get(url, headers=HEADERS)
soup = bs(response.text, "lxml")

table = soup.select_one("table.gall_list")

columns = [th.text.strip() for th in table.select("th")]
rows = []
for tr in table.select("tbody tr"):
    tds = tr.select("td")

    row = [td.text.strip() for td in tds]
    if row[1] == "일반" : # 일반 게시물만 크롤링
        rows.append(row)
    else :
        continue
    
# DataFrame 생성
stock_table = pd.DataFrame(rows, columns=columns)

print(len(stock_table))
stock_table.head()

50


,번호,말머리,제목,글쓴이,작성일,조회,추천
0,4325356,일반,양자컴 나오면 즉시 뒤지는 애들,찢황만세(1.235),24.12.11,95,3
1,4325355,일반,찢 깜빵가냐?\n[2],ㅇㅇ(118.235),24.12.11,66,0
2,4325354,일반,형두창단조부활ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,HNH,24.12.11,45,0
3,4325353,일반,HBM 퀼통과해도 삼전 주가 구데기일듯\n[2],ㅇㅇ(223.38),24.12.11,88,0
4,4325352,일반,신성 평단 105000 오늘 다 정리했다,찢황만세(211.235),24.12.11,277,0


# 2. 페이지 구간에서 row 추출 테스트

## 2.1 목록에서 링크 통해 for문 안에서 게시글 내용까지 접근 (사용 X)

In [ ]:
# 코스피갤
gal_id = "kospi"

# 조회할 페이지
start_page = 1
end_page = 10

posts = {
    "post_id" : [], # 게시글 id
    "subject" : [], # 말머리(분류)
    "title" : [], # 제목
    "date" : [], # 작성일
    "view_count" : [], # 조회수
    "recommend_count" : [], # 추천수
    "content" : [] # 게시글 내용
}

for page_no in range(start_page, end_page+1, 1) :
    url = f"https://gall.dcinside.com/mgallery/board/lists/?id={gal_id}&page={page_no}"
    
    response = requests.get(url, headers=HEADERS)
    soup = bs(response.text, "lxml")

    table = soup.select_one("table.gall_list")

    for tr in table.select("tbody tr"): # tr : 게시글 목록에서 한 row
        # tds = tr.select("td") # td(s) : 번호 / 말머리 / 제목 / 글쓴이 / 작성일 / 조회 / 추천
        if tr.select_one("td.gall_subject").text.strip() in ("설문", "AD", "공지") :
            continue
        # if tr.select_one("td.gall_date"). not in () # 날짜가 원하는 기간이 아닐 때 continue 또는 아예 크롤링 중단하는 코드 추가해야함
        
        posts["subject"].append(tr.select_one("td.gall_subject").text.strip())
        
        posts["post_id"].append(tr.select_one("td.gall_num").text.strip())
        posts["title"].append(tr.select_one("td.gall_tit a").text.strip())
        posts["date"].append(tr.select_one("td.gall_date").text.strip())
        posts["view_count"].append(tr.select_one("td.gall_count").text.strip())
        posts["recommend_count"].append(tr.select_one("td.gall_recommend").text.strip())
        
        # 제목에 달린 링크로 게시글 가져오기
        post_link = tr.select_one("td.gall_tit a").attrs["href"]
        post_url = f"https://gall.dcinside.com{post_link}"
        post_response = requests.get(post_url, headers=HEADERS)
        post_soup = bs(post_response.text, "lxml")
        
        # print(post_soup.select_one("div.write_div"))
        
        posts["content"].append(post_soup.select_one("div.write_div").text.strip())
        
        
df = pd.DataFrame(posts)

print(len(df))
df.head()

df.to_csv("./test_data.csv") # 2분 7초

495


## 2.2 df에 게시글 링크를 저장하는 방식 (사용 O)
- 추후에 게시글 url을 순회하며 게시글 내용을 따로 저장

In [ ]:
# 조회할 페이지 정보
start_page = 1
end_page = 10

posts = {
    "post_id" : [], # 게시글 id
    "subject" : [], # 말머리(분류)
    "title" : [], # 제목
    "date" : [], # 작성일
    "view_count" : [], # 조회수
    "recommend_count" : [], # 추천수
    "post_url" : [] # 게시글 url
}

for page_no in range(start_page, end_page+1, 1) :
    url = f"https://gall.dcinside.com/mgallery/board/lists/?id={gal_id}&page={page_no}"
    
    response = requests.get(url, headers=HEADERS)
    soup = bs(response.text, "lxml")

    table = soup.select_one("table.gall_list")

    for tr in table.select("tbody tr"): # tr : 게시글 목록에서 한 row
        if tr.select_one("td.gall_subject").text.strip() in ("설문", "AD", "공지") :
            continue

        posts["subject"].append(tr.select_one("td.gall_subject").text.strip())
        
        posts["post_id"].append(tr.select_one("td.gall_num").text.strip())
        posts["title"].append(tr.select_one("td.gall_tit a").text.strip())
        posts["date"].append(tr.select_one("td.gall_date").text.strip())
        posts["view_count"].append(tr.select_one("td.gall_count").text.strip())
        posts["recommend_count"].append(tr.select_one("td.gall_recommend").text.strip())
        
        # 제목에 달린 링크 저장
        post_link = tr.select_one("td.gall_tit a").attrs["href"]
        post_url = f"https://gall.dcinside.com{post_link}"
        posts["post_url"].append(post_url)
        
        
df = pd.DataFrame(posts)

print(len(df))
df.head()

df.to_csv("./test_data.csv") # 2.3초

495


# 3. 원하는 기간을 포함하는 모든 게시글 추출 (posts_meta.csv)
- 목록에서 추출할 수 있는 부분만 추출
  - 게시글 id, 말머리, 제목, 작성일, 조회수, 추천수, 게시글 url
- 코스피 자체의 활동지수를 측정할 때 활용 가능

In [ ]:
# Import pacakge
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

# 코스피갤
gal_id = "kospi"

# 접속 정보
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36"
}

# 조회할 페이지 정보
start_page = 1
end_page = 10110

posts = {
    "post_id" : [], # 게시글 id
    "subject" : [], # 말머리(분류)
    "title" : [], # 제목
    "date" : [], # 작성일
    "view_count" : [], # 조회수
    "recommend_count" : [], # 추천수
    "post_url" : [] # 게시글 url
}

for page_no in range(start_page, end_page+1, 1) :
    url = f"https://gall.dcinside.com/mgallery/board/lists/?id={gal_id}&page={page_no}"
    
    response = requests.get(url, headers=HEADERS)
    soup = bs(response.text, "lxml")

    table = soup.select_one("table.gall_list")

    for tr in table.select("tbody tr"): # tr : 게시글 목록에서 한 row
        if tr.select_one("td.gall_subject").text.strip() in ("설문", "AD", "공지") :
            continue

        posts["subject"].append(tr.select_one("td.gall_subject").text.strip())
        
        posts["post_id"].append(tr.select_one("td.gall_num").text.strip())
        posts["title"].append(tr.select_one("td.gall_tit a").text.strip())
        posts["date"].append(tr.select_one("td.gall_date").text.strip())
        posts["view_count"].append(tr.select_one("td.gall_count").text.strip())
        posts["recommend_count"].append(tr.select_one("td.gall_recommend").text.strip())
        
        # 제목에 달린 링크 저장
        post_link = tr.select_one("td.gall_tit a").attrs["href"]
        post_url = f"https://gall.dcinside.com{post_link}"
        posts["post_url"].append(post_url)
        
    if page_no%100 ==0 :
        print(f"{page_no}페이지 크롤링 완료")
        
df = pd.DataFrame(posts)

print(len(df))
display(df.head())

df.to_csv("./posts_meta.csv")

100페이지 크롤링 완료
200페이지 크롤링 완료
300페이지 크롤링 완료
400페이지 크롤링 완료
500페이지 크롤링 완료
600페이지 크롤링 완료
700페이지 크롤링 완료
800페이지 크롤링 완료
900페이지 크롤링 완료
1000페이지 크롤링 완료
1100페이지 크롤링 완료
1200페이지 크롤링 완료
1300페이지 크롤링 완료
1400페이지 크롤링 완료
1500페이지 크롤링 완료
1600페이지 크롤링 완료
1700페이지 크롤링 완료
1800페이지 크롤링 완료
1900페이지 크롤링 완료
2000페이지 크롤링 완료
2100페이지 크롤링 완료
2200페이지 크롤링 완료
2300페이지 크롤링 완료
2400페이지 크롤링 완료
2500페이지 크롤링 완료
2600페이지 크롤링 완료
2700페이지 크롤링 완료
2800페이지 크롤링 완료
2900페이지 크롤링 완료
3000페이지 크롤링 완료
3100페이지 크롤링 완료
3200페이지 크롤링 완료
3300페이지 크롤링 완료
3400페이지 크롤링 완료
3500페이지 크롤링 완료
3600페이지 크롤링 완료
3700페이지 크롤링 완료
3800페이지 크롤링 완료
3900페이지 크롤링 완료
4000페이지 크롤링 완료
4100페이지 크롤링 완료
4200페이지 크롤링 완료
4300페이지 크롤링 완료
4400페이지 크롤링 완료
4500페이지 크롤링 완료
4600페이지 크롤링 완료
4700페이지 크롤링 완료
4800페이지 크롤링 완료
4900페이지 크롤링 완료
5000페이지 크롤링 완료
5100페이지 크롤링 완료
5200페이지 크롤링 완료
5300페이지 크롤링 완료
5400페이지 크롤링 완료
5500페이지 크롤링 완료
5600페이지 크롤링 완료
5700페이지 크롤링 완료
5800페이지 크롤링 완료
5900페이지 크롤링 완료
6000페이지 크롤링 완료
6100페이지 크롤링 완료
6200페이지 크롤링 완료
6300페이지 크롤링 완료
6400페이지 크롤링 완료
6500페이지 크롤링 완료
6600페이지 크롤링 완료
6700페이지 크롤링 완료
6800

# 4. posts_meta 에서 원하는 키워드가 포함된 제목의 데이터만 추출
- posts_meta에서 데이터 수가 50만개가 넘어서 모든 게시글 url 접근 사실상 불가
- 제목에 키워드가 포함된 데이터만 추출

In [8]:
import pandas as pd
import time
from bs4 import BeautifulSoup as bs

meta_df = pd.read_csv("./posts_meta.csv")

TITLE_KEYWORDS = ["삼전", "삼성전자", "하이닉스", "하닉", "현대차", "현대자동차", "현차"]

candidate_df = meta_df[
    (meta_df["view_count"] >= 100) &
    (
        meta_df["title"].str.contains("|".join(TITLE_KEYWORDS))
    )
]

df = pd.DataFrame(candidate_df)

print(len(df))
display(df.head())

df.to_csv("./posts_three.csv", index=False)


9246


,Unnamed: 0,post_id,subject,title,date,view_count,recommend_count,post_url
398,398,5017415,일반,jp모간 삼전하닉 목표가 올렸놐ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,18:31,103,0,https://gall.dcinside.com/mgallery/board/view/...
472,472,5017331,소식📰,삼전 풀매수 들어갑니다ㅋㅋ,17:30,172,0,https://gall.dcinside.com/mgallery/board/view/...
486,486,5017315,일반,현대차 설겆이 각인데,17:19,119,0,https://gall.dcinside.com/mgallery/board/view/...
588,588,5017214,일반,개인이 현대차를 저정도로 올릴수가있는거엿음?,16:10,100,0,https://gall.dcinside.com/mgallery/board/view/...
725,725,5017047,일반,삼전보니깐 코스피5천찍으묜 곱버스 풀매수해야하는 이유가 보이네,14:55,108,1,https://gall.dcinside.com/mgallery/board/view/...


# 5. 게시글 추출
- posts_three.csv파일에서 url로 게시물 내용 가져오기
- 제목이나 게시글 내용에 대장주 키워드가 포함돼있으면 해당 주식의 df에 추가

In [ ]:
# ==============================
# 1. 키워드 설정
# ==============================

STOCK_KEYWORDS = {
    "삼성전자": ["삼전", "삼성전자"],
    "SK하이닉스": ["하이닉스", "하닉"],
    "현대차": ["현대차", "현대자동차", "현차"]
}

# ==============================
# 2. 이미 필터링된 데이터 로드
# ==============================
df = pd.read_csv("./posts_three.csv")

# ==============================
# 3. 게시물 본문 크롤링
# ==============================
def fetch_content(url):
    try:
        res = requests.get(url, headers=HEADERS, timeout=5)
        soup = bs(res.text, "lxml")

        body = soup.select(".write_div")
        texts = [b.get_text(" ", strip=True) for b in body]
        return " ".join(texts)

    except:
        return ""

df["content"] = ""

for idx, row in tqdm(df.iterrows(), total=len(df)):
    df.at[idx, "content"] = fetch_content(row["post_url"])
    time.sleep(0.3)

# ==============================
# 4. 종목 분류
# ==============================
def classify_stock(row):
    text = f"{row['title']} {row['content']}"
    matched = []

    for stock, keywords in STOCK_KEYWORDS.items():
        for kw in keywords:
            if kw in text:
                matched.append(stock)
                break

    return matched

df["stocks"] = df.apply(classify_stock, axis=1)

# ==============================
# 5. 종목별 분리
# ==============================
samsung_df = df[df["stocks"].apply(lambda x: "삼성전자" in x)]
hynix_df   = df[df["stocks"].apply(lambda x: "SK하이닉스" in x)]
hyundai_df = df[df["stocks"].apply(lambda x: "현대차" in x)]

# ==============================
# 6. 저장
# ==============================
df.to_csv("./posts_all_tagged.csv", index=False)
samsung_df.to_csv("./posts_samsung.csv", index=False)
hynix_df.to_csv("./posts_hynix.csv", index=False)
hyundai_df.to_csv("./posts_hyundai.csv", index=False)

print("완료")


100%|██████████| 9246/9246 [2:03:59<00:00,  1.24it/s]   


완료


# 6. 댓글수 추출
- 게시글 추출할 때 같이 추출하는 거 깜빡..
- 댓글수만 추출해서 posts_three.csv에 댓글수 컬럼 붙여서 새로 저장

In [7]:
df = pd.read_csv("./posts_three.csv")

# url 통해서 댓글수 가져오는 함수
def get_comment_count(post_url):
    res = requests.get(post_url, headers=HEADERS, timeout=10)
    soup = bs(res.text, "lxml")

    comment_span = soup.select_one("span.gall_comment a")
    if not comment_span:
        return 0

    text = comment_span.get_text(strip=True)  # "댓글 20"
    count = re.sub(r"[^\d]", "", text)         # 숫자만 남김

    return int(count) if count else 0

# 댓글 수 수집 루프
comment_counts = []

for idx, row in df.iterrows():
    post_url = row["post_url"]

    try:
        cnt = get_comment_count(post_url)
        comment_counts.append(cnt)

        if idx % 100 == 0:
            print(f"{idx}개 게시글 댓글수 수집 완료")

        time.sleep(0.2)  # 가벼운 요청이므로 짧게

    except Exception as e:
        print(f"[댓글 수 실패] {post_url} | {e}")
        comment_counts.append(None)


df["comment_count"] = comment_counts

print(df[["post_id", "title", "comment_count"]].head())

df.to_csv("./posts_three_with_comment_count.csv", index=False)


0개 게시글 댓글 수집 완료
100개 게시글 댓글 수집 완료
200개 게시글 댓글 수집 완료
300개 게시글 댓글 수집 완료
400개 게시글 댓글 수집 완료
500개 게시글 댓글 수집 완료
600개 게시글 댓글 수집 완료
700개 게시글 댓글 수집 완료
800개 게시글 댓글 수집 완료
900개 게시글 댓글 수집 완료
1000개 게시글 댓글 수집 완료
1100개 게시글 댓글 수집 완료
1200개 게시글 댓글 수집 완료
1300개 게시글 댓글 수집 완료
1400개 게시글 댓글 수집 완료
1500개 게시글 댓글 수집 완료
1600개 게시글 댓글 수집 완료
1700개 게시글 댓글 수집 완료
1800개 게시글 댓글 수집 완료
1900개 게시글 댓글 수집 완료
2000개 게시글 댓글 수집 완료
2100개 게시글 댓글 수집 완료
2200개 게시글 댓글 수집 완료
2300개 게시글 댓글 수집 완료
2400개 게시글 댓글 수집 완료
2500개 게시글 댓글 수집 완료
2600개 게시글 댓글 수집 완료
2700개 게시글 댓글 수집 완료
2800개 게시글 댓글 수집 완료
2900개 게시글 댓글 수집 완료
3000개 게시글 댓글 수집 완료
3100개 게시글 댓글 수집 완료
3200개 게시글 댓글 수집 완료
3300개 게시글 댓글 수집 완료
3400개 게시글 댓글 수집 완료
3500개 게시글 댓글 수집 완료
3600개 게시글 댓글 수집 완료
3700개 게시글 댓글 수집 완료
[댓글 수 실패] https://gall.dcinside.com/mgallery/board/view/?id=kospi&no=4824755&page=3187 | HTTPSConnectionPool(host='gall.dcinside.com', port=443): Read timed out. (read timeout=10)
3800개 게시글 댓글 수집 완료
[댓글 수 실패] https://gall.dcinside.com/mgallery/board/view/?id=kospi&no=4821164&page=3245 | HT

In [9]:
# 댓글수 실수값으로 저장됨 -> 정수로 수정
df = pd.read_csv("./posts_three_with_comment_count.csv")

df["comment_count"] = (
    df["comment_count"]
    .fillna(0)        # NaN → 0
    .astype(int)      # int 변환
)

df.to_csv(
    "./posts_three_with_comment_count_int.csv",
    index=False,
    encoding="utf-8-sig"
)

In [11]:
import pandas as pd

# 댓글 수 기준 데이터
comment_df = pd.read_csv("./posts_three_with_comment_count_int.csv")

comment_df = comment_df[["post_id", "comment_count"]]

In [12]:
def add_comment_count(input_path, output_path):
    df = pd.read_csv(input_path)

    df_merged = df.merge(
        comment_df,
        on="post_id",
        how="left"        # 게시글은 유지, 댓글 수만 붙임
    )

    df_merged["comment_count"] = (
        df_merged["comment_count"]
        .fillna(0)
        .astype(int)
    )

    df_merged.to_csv(
        output_path,
        index=False,
        encoding="utf-8-sig"
    )

    print(f"{output_path} 저장 완료")

In [13]:
add_comment_count(
    "./posts_samsung.csv",
    "./posts_samsung_with_comment.csv"
)

add_comment_count(
    "./posts_hynix.csv",
    "./posts_hynix_with_comment.csv"
)

add_comment_count(
    "./posts_hyundai.csv",
    "./posts_hyundai_with_comment.csv"
)

./posts_samsung_with_comment.csv 저장 완료
./posts_hynix_with_comment.csv 저장 완료
./posts_hyundai_with_comment.csv 저장 완료
